<a href="https://colab.research.google.com/github/abhi78945/-Crop-Yield-Prediction/blob/main/Crop_Yield_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Crop Yield Prediction 99%

Step 1 | Import Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import geopandas as gpd
import plotly.express as px

In [ ]:
class colorss:
    yellows=['#ffffd4','#fee391','#fec44f','#fe9929','#d95f0e','#993404','#a70000','#ff5252','#ff7b7b','#ffbaba']
    greens=['#ffffd4','#fee391','#fec44f','#fe9929','#d9f0a3','#addd8e','#78c679','#41ab5d','#238443','#005a32']
cmaps=['flare','icefire','bwr_r','Accent','Spectral','RdGy','afmhot_r','afmhot','inferno','seismic','vlag','vlag_r']

In [ ]:
df=pd.read_csv("/content/yield_df.csv")
df.head()

In [ ]:
df.describe()

Step 3.1 | Dataset Basic Information

In [ ]:
df.info()

Inferences:

Number of Entries:

 The dataset consists of 28242 entries, ranging from index 0 to 28241.

Columns: There are 8 columns in the dataset.

Data Types:

Most of the columns (6 out of 8) are of the int64 & float64 data type.
Only the Item and Area columns are of the object data type.

Missing Values:

There doesn't appear to be any missing values in the dataset as each column has 28242 non-null entries.

Step 3.2 | Summary Statistics for Numerical Variables¶

**Numerical Features:**

**average_rain_fall_mm_per_year:** The average rainfall per year is approximately 1149, with the least rainfall being 51 and the most 3240.

**pesticides_tonnes:** The mean pesticides used in tonnes is a whopping 37077 tonnes, with minimum as little as 0.04 and maximum as huge as 367778 tonnes.

**hg/ha_yield:** The average crop production yield is 77053.3. Ranging from 50 all the way to 501412 hectograms per hectare.

Step 3.3 | Summary Statistics for Categorical Variables

In [ ]:
df.describe(include='object')

**Categorical Features (object data type):**

**Area:** There are 101 unique countries/areas, with India being the most frequent category, occurring 4048 times out of 28242 entries.

**Item:** 10 unique types of crops are present. The most common type is "Potatoes", occurring 4276 times.

Step 4 | EDA

In [ ]:
df.drop("Unnamed: 0", axis=1,inplace=True)
df

In [ ]:
# remove countries with less than 100 record
country_counts =df['Area'].value_counts()
countries_to_drop = country_counts[country_counts < 100].index.tolist()
df_filtered = df[~df['Area'].isin(countries_to_drop)]
df = df_filtered.reset_index(drop=True)

In [ ]:
df

In [ ]:
datacorr=df.copy()

In [ ]:
from sklearn.preprocessing import LabelEncoder
categorical_columns = datacorr.select_dtypes(include=['object']).columns.tolist()
label_encoder = LabelEncoder()
for column in categorical_columns:
    datacorr[column] = label_encoder.fit_transform(datacorr[column])

In [ ]:
sns.heatmap(datacorr.corr() , annot= True , cmap='rocket')

**Inferences:**

**Area:** There's a strong correlation between Area and pesticides_tonnes, along with Area and average rainfall

**Items:** There's a strong correlation between Item and hg/ha_yield

In [ ]:
sns.set(palette='BrBG')
df.hist(figsize=(5,10));

**Inferences:**

**rainfall:** Most rainfall is between 0-1000 and very few is around 3000

**pesticides_tonnes:** The majority of used pesticides is little to zero

**avg_temp:** Most average temperatures is around 25. *hg/ha yield: There is a vast majority of yield production around 0.

In [ ]:
sns.pairplot(data=df,hue='Item',kind='scatter',palette='BrBG')

In [ ]:
df2=df[df['Item']=='Yams']
df2.groupby('Year')['hg/ha_yield'].mean().plot(color='brown')

**Inferences:**

**Items:**

**yams:** It appears that yams uses a lot of pesticides, grow on all rainfall ranges.

**wheat:** Wheat thrives on all average temperature ranges and grows on average to no rainfall.

**year:**

2003-2013: Starting from 2003 to 2013, more and more pesticides were used on yams to increase its productivity.

**hg/ha_yield:** Sorghum and Soybeans are lowest in yield production while potatoes are most popular.

In [ ]:
geojson_url = "https://raw.githubusercontent.com/nvkelso/natural-earth-vector/master/geojson/ne_110m_admin_0_countries.geojson"
data = gpd.read_file(geojson_url)

merged_data = data.merge(df, left_on='NAME', right_on='Area', how='left')

In [ ]:
# fig, ax = plt.subplots(figsize=(15, 10))
merged_data.plot( column='hg/ha_yield', cmap='Greens_r', linewidth=0.8, edgecolor='0.8')
# plt.title("Countries")
plt.show()

In [ ]:
del merged_data

In [ ]:
del data

In [ ]:
# import plotly.io as pio
# pio.renderers.default = 'notebook'

In [ ]:
palette = sns.color_palette('tab20', 21,as_cmap=True)
num_plots = 7
areas_per_plot = 10

# Get unique areas
unique_areas = sorted(df['Area'].unique())

# Split into chunks
area_chunks = [unique_areas[i:i+areas_per_plot] for i in range(0, len(unique_areas), areas_per_plot)]
area_chunks[-2] = unique_areas[-11:]
fig, axs = plt.subplots(ncols=num_plots, figsize=(30, 10))
j=0
for i, ax in enumerate(axs):

    plot_df = df[df['Area'].isin(area_chunks[i])]
    for i, area in enumerate(plot_df['Area'].unique()):
       data = plot_df[plot_df['Area'] == area]
       ax.hist(data['hg/ha_yield'], facecolor=palette(i), label=area)

    ax.legend()
    j+=1

plt.show()

In [ ]:
for i in range(0,7):
    plot_df = df[df['Area'].isin(area_chunks[i])]
    dk=plot_df.groupby(['Area','Item'])['hg/ha_yield'].mean().to_frame()
    dg=dk.sort_values(by=['hg/ha_yield'],ascending=True)
    display(dg.head())

In [ ]:
for i in range(0,7):
    plot_df = df[df['Area'].isin(area_chunks[i])]
    dk=plot_df.groupby(['Area','Item'])['hg/ha_yield'].mean().to_frame()
    dg=dk.sort_values(by=['hg/ha_yield'],ascending=False)
    display(dg.head())

In [ ]:
dk=df.groupby(['Area','Item'])['hg/ha_yield'].mean().to_frame()
dk.sort_values(by=['hg/ha_yield'],ascending=False)

**Inferences:**

**Group 0:** Australia produced the largest amount of yield harvesting potatoes while Angola has the least yield harvesting sorghum,soybeans and maize.

**Group 1:** Egypt was producing most yield in this group growing sweet potatoes and potatoes while Ecuador was struggling in harvesting wheat.

**Group 2:** France and Germany are at the top of yield production both harvesting potatoes while Honduras wasn't in luck harvesting wheat

**Group 3:** India soared in growing Cassava and Japan was at the top growing potatoes while Madagascar struggled to grow soybeans and sorghum.

**Group 4:** Morocco and Mexico shined in yield production of potatoes while Niger failed to have high productions of wheat

**Group 5:** Pakistan was struggling in growing sorghum while South Africa Spain, and Saudi Arabia shined in growing Potatoes

**Group 6:** Tajikistan failed to harvest large amounts of soybeans while United Kingdom and Turkey produced a myriad of potatoes

**Collectively:**
Top producing countries: United Kingdom, France, Germany, Australia, and Japan
Top produced Item: potatoes
Least producing countries: Zimbabwe, Azerbaijan, Angola, Niger, Tajikistan
Least produced item: sorghum and soybeans

In [ ]:
for i in range(0,7):
    plot_df = df[df['Area'].isin(area_chunks[i])]
    plot_df.groupby(['Area'])['average_rain_fall_mm_per_year'].mean().plot(kind='bar',rot=0,color=colorss.greens)
    plt.xticks(rotation=90)
    plt.show()

**Inferences:**

**Top rainfall countries:** Bangladesh, Colombia, Guyana, Indonesia, Nicaragua, Papua New Guinea, Ecuador, and Suriname have rainfall with over 2000 mm

**Least rainfall countries:** Saudi Arabia, Pakistan, South Africa, Mali, Mauritania, Morocco, Niger, Libya, Iraq, Egypt, Azerbaijan,and Algeria have rainfall with less than 500 mm

In [ ]:
for i in range(0,7):
    plot_df = df[df['Area'].isin(area_chunks[i])]
    plot_df.groupby(['Area'])['pesticides_tonnes'].mean().plot(kind='bar',rot=0,color=colorss.yellows)
    plt.xticks(rotation=90)
    plt.show()

**Inferences:**

Top pesticides using countries: Brazil, Italy, Argentina

**France:** For a high yield producing country, using over 80000 tonnes of pesticides could be harmful.

**Japan:** Another high producing country using over 60000 tonnes of pesticides.

**Least pesticides using countries:** Algeria, Angola, Azerbaijan, Bulgaria, Burkina Faso, Burundi, Cameroon, Cenrral African Republic, Croatia, Egypt, El Salvador,Greece, Guinea, Guyana, Haiti, Honduras, Hungary, Indonesia, Iraq, Jamaica, Kenya, Kazakhstan, Libya, Madagascar, Malawi, Mali, Mauritana, Mauritius, Mozambique, Nepal, Niger, Papua New

Guinea, Rwanda, Senegal, Saudi Arabia, Sri Lanka, Suriname, Tajikistan, Uganda, Zambia, Zimbabwe, Uruguay have been using pesticides with less than 10000 tonnes

All poorly producing yield countries are included in least pesticides usage countries

Germany, Australia, and United Kingdom are top producing countries and they use an average amount of pesticides around 30000 tonnes

In [ ]:
for i in range(0,7):
    plot_df = df[df['Area'].isin(area_chunks[i])]
    plot_df.groupby('Area')[['pesticides_tonnes', 'hg/ha_yield']].mean().plot(kind='bar',rot=0,color=colorss.yellows[-6:])
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
px.scatter(df, x='hg/ha_yield', y='pesticides_tonnes',color="Area",color_discrete_sequence=colorss.greens)

In [ ]:
num_plots = 7
areas_per_plot = 10

# Get unique areas
unique_areas = sorted(df['Area'].unique())

# Split into chunks
area_chunks = [unique_areas[i:i+areas_per_plot] for i in range(0, len(unique_areas), areas_per_plot)]
area_chunks[-2] = unique_areas[-11:]
fig, axs = plt.subplots(ncols=num_plots)
j=0
for i, ax in enumerate(axs):

    plot_df = df[df['Area'].isin(area_chunks[i])]

    ax = px.scatter(plot_df, x='hg/ha_yield', y='pesticides_tonnes',color="Area",color_discrete_sequence=colorss.greens)
    j+=1

    ax.show()
plt.clf()

Inferences:
Group 0:
Brazil had low yield production and as it used more and more pesticides their production went higher.

However, Argentina, Australia, and Algeria kept their pesticides usage low and they had more yield production that Brazil ever could.

Group 1:
Colombia kept having low yield production despite using more pesticides. It was almost constant despite
the increased usage of pesticides.
Canada had one high yield production then went lower while using more pesticides until it had the same fate as Colombia
Egypt, El Salvador, Dominican Republic, and Chile on the other hand, got very high yield productions while using a significantly lower amount of pesticides.

Group 2:
France have high yield production despite its high pesticide usage as it is one of the leading yield production countries. Although, it
essened a little with more pesticides usage.

Germany, Guatemala, and Guyana have high yield production while using less pesticides. even Guatemala had higher production than France at some point.

Group 3:
India and Japan have high yield productions while using less amounts of pesticides. India even beat Japan in some yield productions while using less pesticides as well.

Italy however, is one of the countries that has the highest pesticides usage and its yield production wasn't as high as India and Japan
Group 4:
Mexico uses a significant amount of pesticides and its yield production kept increasing as more pesticides were used.

Mali, Mauritius, and Niger had some high yield productions while using almost no pesticides which is impressive.
Morocco kept using slightly more pesticides until it got a soaring 350k+ production then decreased slightly again.
Nicaragua used more pesticides got to a maximum yield production then decreased again to an amount even lesser thaan it used to have before additional pesticides.
Group 5:
Senegal got to a soaring 350k+ production while using almost no pesticides.
Saudi Arabia got high yield productions while using a small amount of pesticides.
Portugal got more yield productions while utilizing a little more pesticides until it reached a certain threshold and decreased again.
South Africa has a steady increase of yield production while adding more and more pesticides
Spain on the other hand, has some yield flunctuations but overall it is a high yield production. Although Senegal and South Africa beat it using less pesticides

Group 6:

Tajikistan and Suriname use almost no pesticides and they reached high amounts of yield productions. Even though Tajikistan is one of the least producing countries but it has very high potential of being close to the top producing countries.
United Kingdom has high yield production and uses a fair amount of pesticides it's no surprise as it's one of the leading producing countries.
Turkey has lesser yield productions than United Kingdom and Suriname while utilizing more pesticides. *Thailand and Ukraine used way more pesticides and still their productions were lower than the above countries

Conclusion:
While using pesticides increases yield production, there was an obvious pattern happening.

Yield producion is at a considerable amomunt -> use pesticides -> yield production increases -> comes to maximum production -> yield gradually decreases -> yield becomes even less than the starting yield amount no matter how many more pesticides the country uses.

However, there were some exceptions to this pattern such as France and Japan as they are top leading producing countries and other countries.
You can increase production by using more pesticides but it is temporary nonetheless and in the long run it is better to use less pesticides.
There were a lot of countries that showed impressive yield productions using almost no pesticides and this amount was never reached by other countries who do use pesticides. Those countries were Senegal, Suriname, Tajikistan, Mali, Mauritius, Guyana, Algeria, and El Salvador. They should focus more on growing more crops and businessmen should invest in growing crops in these countries as they have a very high potential in yield production of crops.
Other countries who use a tiny amount of pesticides and still have promising yield productions are Egypt, Chile, Morocco, South Africa, and Guatemala. It is also recommended to invest in these countries as they have reassuring yield productions.
The perfect amount of pesticides to use without having to go through the tedious pattern is 30k and below.

In [ ]:
sns.barplot(data=df, x = df.Item, y = df['pesticides_tonnes'],palette='BrBG')
plt.xticks(rotation=90)
plt.show()

**Inferences:**

**Top pesticides used on items:** Yams
**Least pesticides used on items:** Plantains and others

In [ ]:
a4_dims = (16.7, 8.27)

fig, ax = plt.subplots(figsize=a4_dims)
sns.boxplot(x="Item",y="hg/ha_yield",palette="BrBG",data=df,ax=ax)

**Inferences:**

Top producing items: potatoes, cassava
Least producing items: soybeans, sorghum, wheat, maize, and rice, paddy

In [ ]:
sns.scatterplot(x = 'Item', y = 'avg_temp', data = df,size=10,color='y')
plt.xticks(rotation=90);

**Inferences:**

Cassava, Yams, and Plantains and others need more than 15 avg temperature to grow
all other items can grow at any temperature range

In [ ]:
grouped = df.groupby('Item')

best_areas = []

for item, group in grouped:
    max_production_row = group[group['hg/ha_yield'] == group['hg/ha_yield'].max()]

    area = max_production_row['Area'].values[0]
    production = max_production_row['hg/ha_yield'].values[0]

    best_areas.append({'Item': item, 'Area': area, 'hg/ha_yield': production})
best_areas_df = pd.DataFrame(best_areas)

best_areas_df

In [ ]:
ax=sns.barplot(data=best_areas_df,x='hg/ha_yield',y='Area',hue='Item',palette=colorss.yellows)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

**Inferences:**

Cassava: India is the top at producing Cassava
Maize: Tajikistan excels at growing Maize
PLantains and others: Guatemala is the best source of PLantains and others
Potatoes: France is the top at producing potatoes
Rice, paddy: Australia is the leading country in producing rice, paddy
Sorghum: Algeria excels at producing sorghum
Soybeans: Turkey is unrivalled in producing soybeans
Sweet potatoes: Senegal is the best at producing sweet potatoes
Wheat: Zambbia surpasses every country at producing wheat
Yams: Mali outshines all countries in growing yams

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(18, 22))

sns.scatterplot(x = "pesticides_tonnes", y = "hg/ha_yield", hue = "Item", data = df, ax=axes[0], legend = True,palette='Spectral')
axes[0].tick_params(axis='x', rotation=45)
axes[0].set_ylabel('Average Yield')
axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

sns.scatterplot(x = "average_rain_fall_mm_per_year", y = "hg/ha_yield", hue = "Item", data = df, ax=axes[1], legend = True,palette='Spectral')
axes[1].tick_params(axis='x', rotation=45)
axes[1].set_ylabel('Average Yield')
axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

sns.scatterplot(x = "avg_temp", y = "hg/ha_yield", hue = "Item", data = df, ax=axes[2], legend = True,palette='Spectral')
axes[2].tick_params(axis='x', rotation=45)
axes[2].set_ylabel('Average Yield')
axes[2].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

sns.lineplot(x = "Year", y = "hg/ha_yield", hue = "Item", data = df, ax=axes[3], legend = True,palette='Spectral')
axes[3].tick_params(axis='x', rotation=45)
axes[3].set_ylabel('Average Yield')
axes[3].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

**Inferences:**

pesticidies_tonnes: it appears that production of items decreases as pesticides increases
rainfall: it appears that production of items decreases as rainfall increases for more than 2500
avg_temp: it shows that average yield decreases as average temperature decreases below 5

In [ ]:
def change_of_years(data, template='seaborn'):
    col = data.columns[3:].tolist()
    for i in col:
        sns.lineplot(data.groupby(['Year'])[i].mean(),color='brown')
        plt.title=f'Effect of Years on the {i}'


        plt.show()
        yield()

In [ ]:
yplot = change_of_years(df)
next(yplot);

In [ ]:
next(yplot);

**Inferences:**

pesticides have been increasing significantly since 2003

In [ ]:
next(yplot);

In [ ]:
next(yplot);

In [ ]:
df_Egypt = df.loc[df['Area'] == 'Egypt']
yplot = change_of_years(df_Egypt)
next(yplot);

**Inferences:**

A significant decrease in pesticides occurred between 1993-2005 then it increased even higher than its starting value

In [ ]:
next(yplot);

In [ ]:
next(yplot);

Step 5 | Model building and evaluation

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor

In [ ]:
X, y = datacorr.drop(labels='hg/ha_yield', axis=1), datacorr['hg/ha_yield']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
results = []

models = [
    ('Linear Regression', LinearRegression()),
    ('Random Forest', RandomForestRegressor(random_state=42)),
    ('Gradient Boost', GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3,random_state=42)),
    ('XGBoost', XGBRegressor(random_state=42)),
    ('KNN',KNeighborsRegressor(n_neighbors=5)),
    ('Decision Tree',DecisionTreeRegressor(random_state=42)),
    ('Bagging Regressor',BaggingRegressor(n_estimators=150, random_state=42))
          ]
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = model.score(X_test, y_test)
    MSE = mean_squared_error(y_test, y_pred)
    R2_score = r2_score(y_test, y_pred)
    results.append((name, accuracy, MSE, R2_score))
    acc = (model.score(X_train , y_train)*100)
    print(f'The accuracy of the {name} Model Train is {acc:.2f}')
    acc =(model.score(X_test , y_test)*100)
    print(f'The accuracy of the  {name} Model Test is {acc:.2f}')
    plt.scatter(y_test, y_pred,s=10,color='#9B673C')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
#     plt.title(f' {name} Evaluation')
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='green', linewidth = 4)
    plt.show()
dff = pd.DataFrame(results, columns=['Model', 'Accuracy', 'MSE', 'R2_score'])
df_styled_best = dff.style.highlight_max(subset=['Accuracy','R2_score'], color='green').highlight_min(subset=['MSE'], color='green').highlight_max(subset=['MSE'], color='red').highlight_min(subset=['Accuracy','R2_score'], color='red')

# df_styled_worst = dff.style.highlight_max(subset=['MSE'], color='red').highlight_min(subset=['Accuracy','R2_score'], color='red')
display(df_styled_best)
# display(df_styled_worst)

Kefold

In [ ]:
results = []

models = [
    ('Linear Regression', LinearRegression()),
    ('Random Forest', RandomForestRegressor(random_state=42)),
    ('Gradient Boost', GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3,random_state=42)),
    ('XGBoost', XGBRegressor(random_state=42)),
    ('KNN',KNeighborsRegressor(n_neighbors=5)),
    ('Decision Tree',DecisionTreeRegressor(random_state=42)),
    ('Bagging Regressor',BaggingRegressor(n_estimators=150, random_state=42))
          ]
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = model.score(X_test, y_test)
    MSE = mean_squared_error(y_test, y_pred)
    MAE = mean_absolute_error(y_test, y_pred)
    MAPE = mean_absolute_percentage_error(y_test, y_pred)
    R2_score = r2_score(y_test, y_pred)
    results.append((name, accuracy, MSE, MAE, MAPE, R2_score))

    print(name)
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True)
    scores = cross_val_score(model, X, y, cv=kf)

    for fold, score in enumerate(scores):
        print(f"Fold {fold+1}: {score}")

    mean_score = np.mean(scores)
    print(f"Mean Score: {mean_score}")
    print('-'*30)


df = pd.DataFrame(results, columns=['Model', 'Accuracy', 'MSE', 'MAE', 'MAPE', 'R2_score'])
df_styled_best = df.style.highlight_max(subset=['Accuracy','R2_score'], color='lightblue').highlight_min(subset=['MSE','MAE','MAPE'], color='lightblue').highlight_max(subset=['MSE','MAE','MAPE'], color='red').highlight_min(subset=['Accuracy','R2_score'], color='red')

display(df_styled_best)